In [ ]:
# Upload kaggle.json
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rahulsanskar28","key":"b595d6fe1e4be57f3a867e28de4d46ea"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
# ██████████████████████████████████████████████████████████████████████████
# DOCUMENT TAMPERING DETECTION — ELA + EfficientNet-B3 (BEST VERSION)
# ██████████████████████████████████████████████████████████████████████████

import os
import time
import glob
import random
import uuid
import numpy as np
import torch
import torch.nn as nn
import timm

from tqdm import tqdm
from PIL import Image, ImageChops, ImageEnhance
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score


In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)


Using device: cuda


In [ ]:
print("\nDownloading CASIA 2.0...")

os.system(
    "kaggle datasets download -d divg07/casia-20-image-tampering-detection-dataset "
    "-p /content/casia --unzip"
)

CASIA_ROOT = "/content/casia"

# Find authentic and tampered folders automatically
au_dirs = glob.glob(f"{CASIA_ROOT}/**/Au*", recursive=True)
tp_dirs = glob.glob(f"{CASIA_ROOT}/**/Tp*", recursive=True)

au_dirs = [d for d in au_dirs if os.path.isdir(d)]
tp_dirs = [d for d in tp_dirs if os.path.isdir(d)]

print("Authentic folders:", au_dirs)
print("Tampered folders:", tp_dirs)



Authentic folders: ['/content/casia/CASIA2/Au']
Tampered folders: ['/content/casia/CASIA2/Tp']


In [ ]:
def collect_images(dirs):
    images = []
    for d in dirs:
        images += glob.glob(f"{d}/**/*.jpg", recursive=True)
        images += glob.glob(f"{d}/**/*.png", recursive=True)
        images += glob.glob(f"{d}/**/*.tif", recursive=True)
    return images

auth_images = collect_images(au_dirs)
tamp_images = collect_images(tp_dirs)

print("Authentic:", len(auth_images))
print("Tampered:", len(tamp_images))

# Balance
min_len = min(len(auth_images), len(tamp_images), 4000)
random.shuffle(auth_images)
random.shuffle(tamp_images)

auth_images = auth_images[:min_len]
tamp_images = tamp_images[:min_len]

# 80/20 split
split = int(0.8 * min_len)

train_samples = [(p, 0) for p in auth_images[:split]] + \
                [(p, 1) for p in tamp_images[:split]]

val_samples   = [(p, 0) for p in auth_images[split:]] + \
                [(p, 1) for p in tamp_images[split:]]

random.shuffle(train_samples)
random.shuffle(val_samples)

print("Train:", len(train_samples))
print("Val:", len(val_samples))


Authentic: 7437
Tampered: 5123
Train: 6400
Val: 1600


In [ ]:
def generate_ela_image(path, quality=90):
    temp_path = f"/content/temp_{uuid.uuid4()}.jpg"
    try:
        original = Image.open(path).convert("RGB")
        original.save(temp_path, "JPEG", quality=quality)
        compressed = Image.open(temp_path)

        ela_image = ImageChops.difference(original, compressed)
        extrema = ela_image.getextrema()
        max_diff = max([ex[1] for ex in extrema])

        scale = 1.0 if max_diff == 0 else min(255.0/max_diff, 10)
        ela_image = ImageEnhance.Brightness(ela_image).enhance(scale)

    finally:
        if os.path.exists(temp_path):
            os.remove(temp_path)

    return ela_image


In [ ]:
class CASIADataset(Dataset):
    def __init__(self, samples, transform):
        self.samples = samples
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]

        rgb = Image.open(path).convert("RGB")
        ela = generate_ela_image(path)

        rgb = self.transform(rgb)
        ela = self.transform(ela)

        combined = torch.cat([rgb, ela], dim=0)
        combined = torch.clamp(combined, -3, 3)

        return combined, torch.tensor(label, dtype=torch.float)


In [ ]:
transform = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.RandomResizedCrop(300),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

train_ds = CASIADataset(train_samples, transform)
val_ds   = CASIADataset(val_samples, transform)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=2)


In [ ]:
model = timm.create_model("efficientnet_b3", pretrained=True)

# Modify first conv to accept 6 channels
old_weights = model.conv_stem.weight.data.clone()

model.conv_stem = nn.Conv2d(
    6,
    model.conv_stem.out_channels,
    kernel_size=3,
    stride=2,
    padding=1,
    bias=False
)

with torch.no_grad():
    model.conv_stem.weight[:, :3] = old_weights
    model.conv_stem.weight[:, 3:] = old_weights

# Freeze early blocks
for name, param in model.named_parameters():
    if "blocks.0" in name or "blocks.1" in name or "blocks.2" in name:
        param.requires_grad = False

model.classifier = nn.Linear(model.classifier.in_features, 1)
model = model.to(DEVICE)


model.safetensors:   0%|          | 0.00/49.3M [00:00<?, ?B/s]

In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=5e-5
)

scaler = torch.amp.GradScaler("cuda")

EPOCHS = 10
best_auc = 0


In [ ]:
for epoch in range(1, EPOCHS+1):

    print(f"\nEpoch {epoch}/{EPOCHS}")

    model.train()
    total_loss = 0

    for imgs, labels in tqdm(train_loader):
        imgs = imgs.to(DEVICE)
        labels = labels.to(DEVICE)

        optimizer.zero_grad()

        with torch.amp.autocast("cuda"):
            logits = model(imgs).squeeze(1)
            loss = criterion(logits, labels)

        if torch.isnan(loss):
            continue

        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()

    # VALIDATION
    model.eval()
    probs_list, preds_list, true_list = [], [], []

    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs = imgs.to(DEVICE)
            labels = labels.to(DEVICE)

            logits = model(imgs).squeeze(1)
            probs = torch.sigmoid(logits)

            probs_list.extend(probs.cpu().numpy())
            preds_list.extend((probs > 0.5).int().cpu().numpy())
            true_list.extend(labels.cpu().numpy())

    acc = accuracy_score(true_list, preds_list)
    f1  = f1_score(true_list, preds_list)
    auc = roc_auc_score(true_list, probs_list)

    print(f"Loss={total_loss/len(train_loader):.4f} "
          f"Acc={acc:.4f} F1={f1:.4f} AUC={auc:.4f}")

    if auc > best_auc:
        best_auc = auc
        torch.save(model.state_dict(), "/content/document_model.pt")
        print("✔ Best model saved")

print("\nFinal Best AUC:", best_auc)



Epoch 1/10


100%|██████████| 400/400 [02:41<00:00,  2.47it/s]


Loss=0.6761 Acc=0.6750 F1=0.6833 AUC=0.7346
✔ Best model saved

Epoch 2/10


100%|██████████| 400/400 [02:15<00:00,  2.95it/s]


Loss=0.6020 Acc=0.7712 F1=0.7710 AUC=0.8520
✔ Best model saved

Epoch 3/10


...

Loss=0.2859 Acc=0.8800 F1=0.8807 AUC=0.9487
✔ Best model saved

Epoch 9/10


100%|██████████| 400/400 [02:11<00:00,  3.04it/s]


Loss=0.2899 Acc=0.8788 F1=0.8827 AUC=0.9457

Epoch 10/10


100%|██████████| 400/400 [02:13<00:00,  3.00it/s]


Loss=0.2639 Acc=0.8706 F1=0.8737 AUC=0.9398

Final Best AUC: 0.948725


In [ ]:
# =========================
# DOCUMENT TEST EVALUATION ONLY
# =========================

import torch
import numpy as np
from tqdm import tqdm
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    classification_report
)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)

MODEL_PATH = "/content/document_model.pt"

# ─────────────────────────────────────────
# 1️⃣ RECREATE MODEL ARCHITECTURE
# ─────────────────────────────────────────

import timm

model = timm.create_model("efficientnet_b3", pretrained=False)

# Modify first conv to accept 6 channels
model.conv_stem = torch.nn.Conv2d(
    6,
    model.conv_stem.out_channels,
    kernel_size=3,
    stride=2,
    padding=1,
    bias=False
)

model.classifier = torch.nn.Linear(model.classifier.in_features, 1)

model = model.to(DEVICE)

# Load trained weights
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
model.eval()

print("Model loaded successfully ✓")

# ─────────────────────────────────────────
# 2️⃣ BUILD TEST DATASET
# ─────────────────────────────────────────

# Using last 20% as test (same logic as training)
n = len(train_samples + val_samples)
test_samples = (train_samples + val_samples)[int(0.8 * n):]

test_files = [s[0] for s in test_samples]
test_labels = [s[1] for s in test_samples]

test_ds = CASIADataset(test_files, test_labels, val_transform)
test_loader = DataLoader(test_ds, batch_size=16, shuffle=False, num_workers=2)

print("Test samples:", len(test_ds))

# ─────────────────────────────────────────
# 3️⃣ RUN INFERENCE
# ─────────────────────────────────────────

all_probs = []
all_labels = []

with torch.no_grad():
    for imgs, labels in tqdm(test_loader):
        imgs = imgs.to(DEVICE)
        logits = model(imgs).squeeze(1)
        probs = torch.sigmoid(logits)

        all_probs.extend(probs.cpu().numpy())
        all_labels.extend(labels.numpy())

all_probs = np.array(all_probs)
all_labels = np.array(all_labels)
all_preds = (all_probs >= 0.5).astype(int)

# ─────────────────────────────────────────
# 4️⃣ METRICS
# ─────────────────────────────────────────

print("\n" + "═"*60)
print("DOCUMENT TAMPERING — TEST RESULTS")
print("═"*60)

print("Accuracy :", accuracy_score(all_labels, all_preds))
print("Precision:", precision_score(all_labels, all_preds))
print("Recall   :", recall_score(all_labels, all_preds))
print("F1 Score :", f1_score(all_labels, all_preds))
print("AUC      :", roc_auc_score(all_labels, all_probs))

print("\nConfusion Matrix:")
print(confusion_matrix(all_labels, all_preds))

print("\nClassification Report:")
print(classification_report(all_labels, all_preds,
                            target_names=["Authentic","Tampered"]))


Using device: cuda
Model loaded successfully ✓
Test samples: 1600


100%|██████████| 100/100 [00:28<00:00,  3.57it/s]


════════════════════════════════════════════════════════════
DOCUMENT TAMPERING — TEST RESULTS
════════════════════════════════════════════════════════════
Accuracy : 0.676875
Precision: 0.6543075245365322
Recall   : 0.75
F1 Score : 0.6988934187536401
AUC      : 0.6995562500000001

Confusion Matrix:
[[483 317]
 [200 600]]

Classification Report:
              precision    recall  f1-score   support

   Authentic       0.71      0.60      0.65       800
    Tampered       0.65      0.75      0.70       800

    accuracy                           0.68      1600
   macro avg       0.68      0.68      0.68      1600
weighted avg       0.68      0.68      0.68      1600



In [ ]:
!pip install timm -q
